<p>- indetifing customers who have high and medium 'customer life time value'(cltv) is of paramount importance for any business growth, in order to launch new services and advertise existing once to them.<br>
- Not indentifing the high cltv customers correctly will cost the company most, so model with minimum total cost will be selected. The fomula for Total Cost is given as follows.<br>
                        Total Cost = 60*cost_1 + 35*cost_2 +5*cost_3<br>
                   where,<br>
                          cost_1 = false high cltv identifiaction<br>
                          cost_2 = false medium cltv identifiaction<br>
                          cost_3 = false low cltv identifiaction</p>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import PowerTransformer
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve,confusion_matrix


from sklearn import metrics 
from sklearn.model_selection import  train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler
from sklearn.compose import ColumnTransformer

warnings.filterwarnings("ignore")

In [61]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import RobustScaler

In [4]:
train=pd.read_csv('train_2.csv')
test=pd.read_csv('test.csv')
balanced=pd.read_csv('balanced_trainset.csv')

### Functions for model training and evolution

In [47]:
def evaluate_clf(true, predicted):
    '''
    This function takes in true values and predicted values
    Returns: Accuracy, F1-Score, Precision, Recall, Roc-auc Score
    '''
    acc = accuracy_score(true, predicted) # Calculate Accuracy
    f1 = f1_score(true, predicted,average='micro') # Calculate F1-score
    precision = precision_score(true, predicted,average='micro') # Calculate Precision
    recall = recall_score(true, predicted,average='micro')  # Calculate Recall
#     roc_auc = roc_auc_score(true, predicted,average=None,multi_class="ovr") #Calculate Roc
    return acc, f1 , precision, recall

In [23]:
# Create cost of the model as per data description
def total_cost(y_true, y_pred):
    '''
    This function takes y_ture, y_predicted, and prints Total cost due to misclassification
   
    '''
    h1,h2,h3,m1,m2,m3,l1,l2,l3 = confusion_matrix(y_true, y_pred).ravel()
    cost = 60*(h2+h3) + 35*(m1+m2+m3) +5*(l2+l3)
    return cost

In [64]:
# train.drop(['Unnamed: 0'],axis=1,inplace=True)
# balanced.drop(['Unnamed: 0'],axis=1,inplace=True)
test.drop(['Unnamed: 0'],axis=1,inplace=True)
test

,gender,area,qualification,income,num_policies,policy,marital_status,vintage,claim_amount,health_insurance,investment_plans,term_insurance,life_time_value
0,1.070184,1.118853,1.091120,1.059526,1.366832,1.092397,0,2,6376,0,0,1,1
1,1.106625,1.118853,1.091120,1.059526,1.366832,1.092397,1,3,2996,0,1,1,1
2,1.070184,1.118853,1.091120,1.059526,0.443566,1.092397,1,3,4981,1,0,0,0
3,1.106625,1.118853,1.103675,1.059526,1.366832,1.092397,0,7,2532,1,0,0,1
4,1.070184,1.118853,1.103675,1.157693,1.366832,1.092397,1,3,7828,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14744,1.106625,1.118853,1.091120,1.059526,1.366832,1.092397,0,6,4263,1,1,1,1
14745,1.070184,1.118853,1.091120,1.157693,0.443566,1.092397,0,3,5876,0,1,0,0
14746,1.070184,1.118853,1.103675,1.059526,1.366832,1.092397,1,7,4230,0,1,1,2
14747,1.070184,1.118853,1.091120,1.157693,0.443566,1.092397,1,6,5152,0,1,0,1


In [66]:
scaler = RobustScaler()

train.claim_amount = scaler.fit_transform((train.claim_amount.values).reshape(-1, 1))
test.claim_amount = scaler.transform(test.claim_amount.values.reshape(-1, 1))
test

,gender,area,qualification,income,num_policies,policy,marital_status,vintage,claim_amount,health_insurance,investment_plans,term_insurance,life_time_value
0,1.070184,1.118853,1.091120,1.059526,1.366832,1.092397,0,2,0.551927,0,0,1,1
1,1.106625,1.118853,1.091120,1.059526,1.366832,1.092397,1,3,-0.616209,0,1,1,1
2,1.070184,1.118853,1.091120,1.059526,0.443566,1.092397,1,3,0.069812,1,0,0,0
3,1.106625,1.118853,1.103675,1.059526,1.366832,1.092397,0,7,-0.776568,1,0,0,1
4,1.070184,1.118853,1.103675,1.157693,1.366832,1.092397,1,3,1.053741,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14744,1.106625,1.118853,1.091120,1.059526,1.366832,1.092397,0,6,-0.178331,1,1,1,1
14745,1.070184,1.118853,1.091120,1.157693,0.443566,1.092397,0,3,0.379126,0,1,0,0
14746,1.070184,1.118853,1.103675,1.059526,1.366832,1.092397,1,7,-0.189736,0,1,1,2
14747,1.070184,1.118853,1.091120,1.157693,0.443566,1.092397,1,6,0.128910,0,1,0,1


In [59]:
# Create a function which can evaluate models and return a report 
def evaluate_models(train, test, models):
    '''
    This function takes in X and y and models dictionary as input
    It splits the data into Train Test split
    Iterates through the given model dictionary and evaluates the metrics
    Returns: Dataframe which contains report of all models metrics with cost
    '''
    X_train= train.drop(['life_time_value'],axis=1)
    X_test= test.drop(['life_time_value'],axis=1)
    y_train= train['life_time_value']
    y_test = test['life_time_value']
    
    
    cost_list=[]
    models_list = []
    accuracy_list = []
    
    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, y_train) # Train model

        # Make predictions
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Training set performance
        model_train_accuracy, model_train_f1,model_train_precision,\
        model_train_recall=evaluate_clf(y_train ,y_train_pred)
        train_cost = total_cost(y_train, y_train_pred)


        # Test set performance
        model_test_accuracy,model_test_f1,model_test_precision,\
        model_test_recall=evaluate_clf(y_test, y_test_pred)
        test_cost = total_cost(y_test, y_test_pred)

        print(list(models.keys())[i])
        models_list.append(list(models.keys())[i])

        print('Model performance for Training set')
        print("- Accuracy: {:.4f}".format(model_train_accuracy))
        print('- F1 score: {:.4f}'.format(model_train_f1)) 
        print('- Precision: {:.4f}'.format(model_train_precision))
        print('- Recall: {:.4f}'.format(model_train_recall))
#         print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))
        print(f'- COST: {train_cost}.')

        print('----------------------------------')

        print('Model performance for Test set')
        print('- Accuracy: {:.4f}'.format(model_test_accuracy))
        print('- F1 score: {:.4f}'.format(model_test_f1))
        print('- Precision: {:.4f}'.format(model_test_precision))
        print('- Recall: {:.4f}'.format(model_test_recall))
#         print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))
        print(f'- COST: {test_cost}.')
        cost_list.append(test_cost)
        print('='*35)
        print('\n')
        
    report=pd.DataFrame(list(zip(models_list, cost_list)), columns=['Model Name', 'Cost']).sort_values(by=["Cost"])
        
    return report

In [14]:
# Dictionary which contains models for experiment
models = {
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Logistic Regression": LogisticRegression(),
     "K-Neighbors Classifier": KNeighborsClassifier(),
    "XGBClassifier": XGBClassifier(), 
    "AdaBoost Classifier": AdaBoostClassifier()
}

In [48]:
# imbalance dataset evaluation
report=evaluate_models(train, test, models)

Random Forest
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- COST: 1093810.
----------------------------------
Model performance for Test set
- Accuracy: 0.5419
- F1 score: 0.5419
- Precision: 0.5419
- Recall: 0.5419
- COST: 330950.


Decision Tree
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- COST: 1093810.
----------------------------------
Model performance for Test set
- Accuracy: 0.4865
- F1 score: 0.4865
- Precision: 0.4865
- Recall: 0.4865
- COST: 360640.


Gradient Boosting
Model performance for Training set
- Accuracy: 0.5860
- F1 score: 0.5860
- Precision: 0.5860
- Recall: 0.5860
- COST: 1205575.
----------------------------------
Model performance for Test set
- Accuracy: 0.5790
- F1 score: 0.5790
- Precision: 0.5790
- Recall: 0.5790
- COST: 300630.


Logistic Regression
Model performance for Training set
- Accuracy: 0.4782
- F1 score: 0.4782
- P

In [67]:
# balanced dataset evaluation
report=evaluate_models(balanced, test, models)

Random Forest
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- COST: 1149160.
----------------------------------
Model performance for Test set
- Accuracy: 0.5134
- F1 score: 0.5134
- Precision: 0.5134
- Recall: 0.5134
- COST: 337630.


Decision Tree
Model performance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- COST: 1149160.
----------------------------------
Model performance for Test set
- Accuracy: 0.4827
- F1 score: 0.4827
- Precision: 0.4827
- Recall: 0.4827
- COST: 360910.


Gradient Boosting
Model performance for Training set
- Accuracy: 0.6275
- F1 score: 0.6275
- Precision: 0.6275
- Recall: 0.6275
- COST: 1390170.
----------------------------------
Model performance for Test set
- Accuracy: 0.5489
- F1 score: 0.5489
- Precision: 0.5489
- Recall: 0.5489
- COST: 298370.


Logistic Regression
Model performance for Training set
- Accuracy: 0.6189
- F1 score: 0.6189
- P